# sa_fast_v2 with Valid Baseline

Run sa_fast_v2 on the valid baseline (0 overlaps) with REPAIR strategy.

In [1]:
import numpy as np
import pandas as pd
import subprocess
import os
import shutil
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree

getcontext().prec = 25
scale_factor = Decimal('1e15')

WORK_DIR = '/home/code/experiments/005_sa_valid_baseline'
os.makedirs(WORK_DIR, exist_ok=True)
VALID_BASELINE = '/home/code/submission_candidates/candidate_003.csv'

In [2]:
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

In [3]:
def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def get_side_length(trees):
    if not trees:
        return Decimal('0')
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    minx = Decimal(bounds[0]) / scale_factor
    miny = Decimal(bounds[1]) / scale_factor
    maxx = Decimal(bounds[2]) / scale_factor
    maxy = Decimal(bounds[3]) / scale_factor
    return max(maxx - minx, maxy - miny)

def calculate_score(side_lengths):
    score = Decimal('0')
    for n, side in side_lengths.items():
        score += side ** 2 / Decimal(str(n))
    return float(score)

def load_submission(csv_path):
    df = pd.read_csv(csv_path)
    df['x'] = df['x'].astype(str).str.lstrip('s')
    df['y'] = df['y'].astype(str).str.lstrip('s')
    df['deg'] = df['deg'].astype(str).str.lstrip('s')
    
    tree_lists = {}
    side_lengths = {}
    
    for n in range(1, 201):
        prefix = f'{n:03d}_'
        group = df[df['id'].str.startswith(prefix)]
        trees = [ChristmasTree(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
        tree_lists[n] = trees
        side_lengths[n] = get_side_length(trees)
    
    return tree_lists, side_lengths

In [4]:
# Load and verify valid baseline
print("Loading valid baseline...")
baseline_trees, baseline_sides = load_submission(VALID_BASELINE)
baseline_score = calculate_score(baseline_sides)
print(f"Baseline score: {baseline_score:.6f}")

# Verify 0 overlaps
overlap_count = sum(1 for n in range(1, 201) if has_overlap(baseline_trees[n]))
print(f"Overlap count: {overlap_count}")

if overlap_count > 0:
    print("WARNING: Baseline has overlaps!")
    overlapping_n = [n for n in range(1, 201) if has_overlap(baseline_trees[n])]
    print(f"Overlapping N values: {overlapping_n[:10]}...")
else:
    print("Baseline is VALID (0 overlaps)")

Loading valid baseline...


Baseline score: 70.627582


Overlap count: 0
Baseline is VALID (0 overlaps)


In [5]:
# Run sa_fast_v2 with multiple seeds
print("\nRunning sa_fast_v2 with multiple seeds...")

current_best_trees = baseline_trees.copy()
current_best_sides = baseline_sides.copy()
current_best_score = baseline_score

seeds = [42, 123, 456, 789, 1234, 5678, 9999, 11111, 22222]
total_improvements = 0

for seed in seeds:
    print(f"\n--- Seed {seed} ---")
    
    # Save current best as input
    rows = []
    for n in range(1, 201):
        trees = current_best_trees[n]
        for t_idx, tree in enumerate(trees):
            rows.append({
                'id': f'{n:03d}_{t_idx}',
                'x': f's{float(tree.center_x):.12f}',
                'y': f's{float(tree.center_y):.12f}',
                'deg': f's{float(tree.angle):.12f}'
            })
    pd.DataFrame(rows).to_csv(os.path.join(WORK_DIR, 'submission1.csv'), index=False)
    
    # Run sa_fast_v2
    os.chdir(WORK_DIR)
    try:
        result = subprocess.run(
            ['/home/code/sa_fast_v2'],
            capture_output=True,
            text=True,
            timeout=300,
            env={**os.environ, 'SEED': str(seed)}
        )
    except subprocess.TimeoutExpired:
        print(f"  Timeout for seed {seed}")
        continue
    
    # Load and repair
    try:
        opt_trees, opt_sides = load_submission(os.path.join(WORK_DIR, 'submission2.csv'))
    except:
        print(f"  Failed to load output for seed {seed}")
        continue
    
    iter_improvements = 0
    for n in range(1, 201):
        if not has_overlap(opt_trees[n]) and opt_sides[n] < current_best_sides[n]:
            improvement = float(current_best_sides[n] - opt_sides[n])
            current_best_trees[n] = opt_trees[n]
            current_best_sides[n] = opt_sides[n]
            iter_improvements += 1
            total_improvements += 1
            if improvement > 0.0001:
                print(f"  N={n}: improved by {improvement:.6f}")
    
    new_score = calculate_score(current_best_sides)
    print(f"  Improvements: {iter_improvements}, Score: {new_score:.6f} (delta: {current_best_score - new_score:.6f})")
    current_best_score = new_score

print(f"\n=== Final Results ===")
print(f"Total improvements: {total_improvements}")
print(f"Final score: {current_best_score:.6f}")
print(f"Total improvement from baseline: {baseline_score - current_best_score:.6f}")


Running sa_fast_v2 with multiple seeds...

--- Seed 42 ---


  Timeout for seed 42

--- Seed 123 ---


  Timeout for seed 123

--- Seed 456 ---


  Timeout for seed 456

--- Seed 789 ---


  Timeout for seed 789

--- Seed 1234 ---


  Timeout for seed 1234

--- Seed 5678 ---


  Timeout for seed 5678

--- Seed 9999 ---


  Timeout for seed 9999

--- Seed 11111 ---


  Timeout for seed 11111

--- Seed 22222 ---


  Timeout for seed 22222

=== Final Results ===
Total improvements: 0
Final score: 70.627582
Total improvement from baseline: 0.000000


In [ ]:
# Verify final result has 0 overlaps
print("\nVerifying final result...")
final_overlap_count = sum(1 for n in range(1, 201) if has_overlap(current_best_trees[n]))
print(f"Final overlap count: {final_overlap_count}")

if final_overlap_count > 0:
    print("WARNING: Final result has overlaps!")
else:
    print("Final result is VALID (0 overlaps)")

In [ ]:
# Save final result
print("\nSaving final result...")

rows = []
for n in range(1, 201):
    trees = current_best_trees[n]
    for t_idx, tree in enumerate(trees):
        rows.append({
            'id': f'{n:03d}_{t_idx}',
            'x': f's{float(tree.center_x):.12f}',
            'y': f's{float(tree.center_y):.12f}',
            'deg': f's{float(tree.angle):.12f}'
        })

submission_df = pd.DataFrame(rows)
submission_df.to_csv(os.path.join(WORK_DIR, 'submission_final.csv'), index=False)
submission_df.to_csv('/home/submission/submission.csv', index=False)
print(f"Saved submission")
print(f"\nFinal score: {current_best_score:.6f}")
print(f"Baseline score: {baseline_score:.6f}")
print(f"Total improvement: {baseline_score - current_best_score:.6f}")
print(f"Target: 68.896973")
print(f"Gap: {current_best_score - 68.896973:.6f}")